In [39]:
import os
import numpy as np
import python_speech_features as psf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from numpy.lib.arraypad import pad
import librosa
from python_speech_features import mfcc
import seaborn as sns

In [2]:
audio_path = 'G:\Voice Dataset'
audio_paths = [os.path.join(audio_path, f) for f in os.listdir(audio_path)]

In [3]:
def extract_features(audio_path):
    signal, rate = librosa.load(audio_path, sr=16000)
    mfcc_feat = mfcc(signal, rate, numcep=13, nfilt=26, nfft=512)
    return mfcc_feat

In [4]:
def get_labels(audio_path):
    labels = []
    for file in os.listdir(audio_path):
        if file.endswith('.wav'):
            label = file[0]  # Extract the first character of the file name as the label
            if label == 'u':  # If 'u' is present, skip this label
                continue
            if label == 'f':  # If the label is 'f', it is female
                label = 'female'
            elif label == 'm':  # If the label is 'm', it is male
                label = 'male'
            labels.append(label)
    return np.array(labels)

In [5]:
# Extract MFCC features from the audio files
features = [extract_features(audio_path) for audio_path in audio_paths]

# Determine the maximum length of the features
max_len = max([len(f) for f in features])

# Pad features with zeros to the maximum length
X = np.zeros((len(features), max_len, 13))
for i, f in enumerate(features):
    X[i, :len(f), :] = f

In [6]:
X_flat = X.reshape(X.shape[0], -1)

In [7]:
y = get_labels(audio_path)

In [11]:
y.shape

(3842,)

In [12]:
le = LabelEncoder()
y = le.fit_transform(y)

In [13]:
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42,stratify=y)

In [15]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance

In [16]:
xg = xgb.XGBClassifier(n_estimators = 10)
xg.fit(X_train,y_train)
xg_score=xg.score(X_test,y_test)
y_predict=xg.predict(X_test)
y_true=y_test
print('Accuracy of XGBoost: '+ str(xg_score))
precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted') 
print('Precision of XGBoost: '+(str(precision)))
print('Recall of XGBoost: '+(str(recall)))
print('F1-score of XGBoost: '+(str(fscore)))

Accuracy of XGBoost: 0.8972691807542262
Precision of XGBoost: 0.8997074611105136
Recall of XGBoost: 0.8972691807542262
F1-score of XGBoost: 0.8963099697050407


In [20]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']), 
        'max_depth': int(params['max_depth']),
        'learning_rate':  abs(float(params['learning_rate'])),

    }
    clf = xgb.XGBClassifier( **params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = accuracy_score(y_test, y_pred)

    return {'loss':-score, 'status': STATUS_OK }

space = {
    'n_estimators': hp.quniform('n_estimators', 10, 100, 5),
    'max_depth': hp.quniform('max_depth', 4, 100, 1),
    'learning_rate': hp.normal('learning_rate', 0.01, 0.9),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20)
print("XGBoost: Hyperopt estimated optimum {}".format(best))

100%|██████████████████████████████████████████████| 20/20 [52:08<00:00, 156.40s/trial, best loss: -0.9596879063719116]
XGBoost: Hyperopt estimated optimum {'learning_rate': -0.3722009751296153, 'max_depth': 78.0, 'n_estimators': 70.0}


In [44]:
xg = xgb.XGBClassifier(learning_rate= 0.3722009751296153, n_estimators = 70, max_depth =78)
xg.fit(X_train,y_train)
xg_score=xg.score(X_test,y_test)
y_predict=xg.predict(X_test)
y_true=y_test
print('Accuracy of XGBoost: '+ str(xg_score))
precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted') 
print('Precision of XGBoost: '+(str(precision)))
print('Recall of XGBoost: '+(str(recall)))
print('F1-score of XGBoost: '+(str(fscore)))
print(classification_report(y_true,y_predict))
cm=confusion_matrix(y_true,y_predict)

Accuracy of XGBoost: 0.9596879063719116
Precision of XGBoost: 0.9599099175743815
Recall of XGBoost: 0.9596879063719116
F1-score of XGBoost: 0.9595972902570293
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       438
           1       0.97      0.94      0.95       331

    accuracy                           0.96       769
   macro avg       0.96      0.96      0.96       769
weighted avg       0.96      0.96      0.96       769



In [17]:
# Random Forest training and prediction
rf = RandomForestClassifier(random_state = 0)
rf.fit(X_train,y_train) 
rf_score=rf.score(X_test,y_test)
y_predict=rf.predict(X_test)
y_true=y_test
print('Accuracy of RF: '+ str(rf_score))
precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted') 
print('Precision of RF: '+(str(precision)))
print('Recall of RF: '+(str(recall)))
print('F1-score of RF: '+(str(fscore)))
print(classification_report(y_true,y_predict))

Accuracy of RF: 0.9453836150845254
Precision of RF: 0.9466979228249629
Recall of RF: 0.9453836150845254
F1-score of RF: 0.9450845661221878
              precision    recall  f1-score   support

           0       0.93      0.98      0.95       438
           1       0.97      0.90      0.93       331

    accuracy                           0.95       769
   macro avg       0.95      0.94      0.94       769
weighted avg       0.95      0.95      0.95       769



In [33]:
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train,y_train)
svm_score=svm.score(X_test,y_test)
y_predict=svm.predict(X_test)
y_true=y_test
print('Accuracy of SVM: '+ str(svm_score))
precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted') 
print('Precision of SVM: '+(str(precision)))
print('Recall of SVM: '+(str(recall)))
print('F1-score of SVM: '+(str(fscore)))
print(classification_report(y_true,y_predict))
cm=confusion_matrix(y_true,y_predict)

Accuracy of SVM: 0.976592977893368
Precision of SVM: 0.9766598087534368
Recall of SVM: 0.976592977893368
F1-score of SVM: 0.9765655493064047
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       438
           1       0.98      0.96      0.97       331

    accuracy                           0.98       769
   macro avg       0.98      0.98      0.98       769
weighted avg       0.98      0.98      0.98       769



In [34]:
cm

array([[432,   6],
       [ 12, 319]], dtype=int64)